In [26]:
import pandas as pd
import shap
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from matplotlib.pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn import linear_model
rcParams['figure.figsize'] = 15, 6
plt.rcParams['agg.path.chunksize'] = 10000

### datacleaning:
- we first load the dataset

In [27]:
data = pd.read_csv('data/la-haute-borne-data-2017-2020.csv',sep=';')

In [28]:
data.head()

,Wind_turbine_name,Date_time,Ba_avg,Ba_min,Ba_max,Ba_std,Rt_avg,Rt_min,Rt_max,Rt_std,...,Pas_max,Pas_std,Wa_c_avg,Wa_c_min,Wa_c_max,Wa_c_std,Na_c_avg,Na_c_min,Na_c_max,Na_c_std
0,R80721,2017-02-08T08:00:00+01:00,44.990002,44.990002,44.990002,0.0,14.0,14.0,14.0,0.0,...,NaN,NaN,358.04999,NaN,NaN,NaN,358.04999,NaN,NaN,NaN
1,R80721,2017-01-26T02:40:00+01:00,-1.000000,-1.000000,-1.000000,0.0,10.0,10.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R80721,2017-01-26T13:50:00+01:00,-1.000000,-1.000000,-1.000000,0.0,10.0,10.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R80721,2017-01-26T15:00:00+01:00,-1.000000,-1.000000,-1.000000,0.0,10.0,10.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R80721,2017-02-18T01:10:00+01:00,44.990002,44.990002,44.990002,0.0,17.0,17.0,17.0,0.0,...,NaN,NaN,7.99000,NaN,NaN,NaN,7.99000,NaN,NaN,NaN


- then we remove all empty columns

In [29]:
data = data.drop('Va1_avg', 1)
data = data.drop('Va2_avg', 1)
data = data.drop('Pas_avg', 1)

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


In [30]:
data["P_avg"].head()

0     -1.89000
1    197.32001
2    176.45000
3    190.61000
4     -2.88000
Name: P_avg, dtype: float64

- then we clip our windpower, so we do not have negative power

In [31]:
data["P_avg"] = data["P_avg"].clip(lower=0)

In [32]:
data["P_avg"].head()

0      0.00000
1    197.32001
2    176.45000
3    190.61000
4      0.00000
Name: P_avg, dtype: float64

- we have to convert our timestamp to a usefull format

In [33]:
data['Date_time'] = pd.to_datetime(data['Date_time'], utc=True)

data["date"] = pd.to_datetime(data['Date_time']).apply(lambda x: x.date())
data['monthdate'] = pd.DatetimeIndex(data['Date_time']).month
data['year'] = data['Date_time'].dt.year
data['month'] = data['Date_time'].dt.month
data['day'] = data['Date_time'].dt.day
data['year'] = data["year"].values
data['month'] = data["month"].values
data['day'] = data["day"].values

In [34]:
print("days: ",data['day'].head()) 
print("")
print("months: ",data['month'].head()) 
print("")
print("years :",data['year'].head())

days:  0     8
1    26
2    26
3    26
4    18
Name: day, dtype: int64

months:  0    2
1    1
2    1
3    1
4    2
Name: month, dtype: int64

years : 0    2017
1    2017
2    2017
3    2017
4    2017
Name: year, dtype: int64




- first we drop NaNs and delete outliers

In [35]:
selected_columns = data[["Date_time","P_avg","day","Ba_avg","Rt_avg","Yt_avg","Ws_avg","Ot_avg"]]
df = selected_columns.copy()
df.dropna(axis = 0, how ='any', inplace=True)
df = df[df['P_avg'].notna()]
df = df[df['Ba_avg'].notna()]
df = df[df['Rt_avg'].notna()]
df = df[df['Yt_avg'].notna()]
df = df[df['Ws_avg'].notna()]
df = df[df['Ot_avg'].notna()]

In [36]:
Ws = df.sort_values(by=["Ws_avg"], ascending=False) # Wind speed
Ot = df.sort_values(by=["Ot_avg"], ascending=False) # temperature
Yt = df.sort_values(by=["Yt_avg"], ascending=False) # nacelle temperature
Rt = df.sort_values(by=["Rt_avg"], ascending=False) # hub temperature
Ba = df.sort_values(by=["Ba_avg"], ascending=False) # pitch angle

- we look at every column to see whether there appear some untypical numbers or not.

In [37]:
Ws.head() 

,Date_time,P_avg,day,Ba_avg,Rt_avg,Yt_avg,Ws_avg,Ot_avg
212826,2018-01-03 07:20:00+00:00,284.88,3,78.71,18.32,16.30,24.27,8.12
210705,2018-01-03 07:20:00+00:00,545.07,3,66.90,17.92,16.36,23.48,7.76
177441,2018-01-03 07:20:00+00:00,1296.88,3,44.33,19.00,16.07,23.00,7.77
212232,2018-01-03 04:30:00+00:00,2048.26,3,19.92,18.00,19.32,21.67,12.54
173248,2018-01-03 05:20:00+00:00,1436.69,3,38.10,18.77,18.46,21.30,11.39


--> no outliers in Windspeed

In [38]:
Ot.head()

,Date_time,P_avg,day,Ba_avg,Rt_avg,Yt_avg,Ws_avg,Ot_avg
74205,2017-02-03 13:50:00+00:00,4.3,3,75.300003,15.46,39.560001,7.90,70.980003
15679,2017-02-03 13:40:00+00:00,0.0,3,91.699997,15.16,56.090000,6.08,68.699997
108390,2017-06-21 12:00:00+00:00,0.0,21,45.000000,36.00,41.630000,2.70,35.870000
108384,2017-06-21 11:30:00+00:00,0.0,21,45.000000,35.00,40.530000,0.88,35.860000
100836,2017-06-21 12:00:00+00:00,0.0,21,43.910000,35.00,40.530000,1.07,35.700000


--> two significant outliers which we will set to the more natural max of 35

In [39]:
df["Ot_avg"] = df["Ot_avg"].clip(upper=36)

In [40]:
clippedOt = df.sort_values(by=["Ot_avg"], ascending=False) # temperature
clippedOt.head()

,Date_time,P_avg,day,Ba_avg,Rt_avg,Yt_avg,Ws_avg,Ot_avg
15679,2017-02-03 13:40:00+00:00,0.0,3,91.699997,15.16,56.090000,6.08,36.00
74205,2017-02-03 13:50:00+00:00,4.3,3,75.300003,15.46,39.560001,7.90,36.00
108390,2017-06-21 12:00:00+00:00,0.0,21,45.000000,36.00,41.630000,2.70,35.87
108384,2017-06-21 11:30:00+00:00,0.0,21,45.000000,35.00,40.530000,0.88,35.86
100836,2017-06-21 12:00:00+00:00,0.0,21,43.910000,35.00,40.530000,1.07,35.70


In [41]:
Yt.head()

,Date_time,P_avg,day,Ba_avg,Rt_avg,Yt_avg,Ws_avg,Ot_avg
15679,2017-02-03 13:40:00+00:00,0.00,3,91.699997,15.16,56.09,6.08,68.699997
48060,2017-07-06 14:40:00+00:00,52.92,6,-0.710000,34.49,42.79,4.42,32.350000
129799,2017-07-08 07:20:00+00:00,0.00,8,45.000000,32.00,42.79,2.55,30.180000
158329,2017-06-21 13:50:00+00:00,74.15,21,-0.410000,36.00,42.70,4.76,34.370000
205953,2017-08-29 10:20:00+00:00,18.50,29,0.680000,32.96,42.69,4.12,32.460000


In [42]:
Rt.head()

,Date_time,P_avg,day,Ba_avg,Rt_avg,Yt_avg,Ws_avg,Ot_avg
163009,2017-09-07 11:50:00+00:00,0.0,7,91.68,215.00,27.59,4.14,18.22
138342,2017-09-07 12:10:00+00:00,0.0,7,91.68,215.00,27.22,3.86,18.09
40783,2017-09-07 12:00:00+00:00,0.0,7,91.68,215.00,26.87,3.04,18.31
114277,2017-09-07 11:40:00+00:00,0.0,7,91.68,208.00,29.26,4.43,18.12
65624,2017-09-07 12:20:00+00:00,0.0,7,89.05,206.54,27.26,3.36,18.21


In [43]:
Ba.head()

,Date_time,P_avg,day,Ba_avg,Rt_avg,Yt_avg,Ws_avg,Ot_avg
196885,2017-05-31 08:20:00+00:00,0.0,31,132.48,24.98,36.71,2.12,23.63
175114,2017-05-31 06:40:00+00:00,0.0,31,116.29,24.77,29.03,1.28,21.98
199986,2017-09-07 06:50:00+00:00,0.0,7,114.93,22.55,24.47,2.68,14.69
1315,2017-05-30 06:50:00+00:00,0.0,30,111.78,28.00,30.00,6.17,22.00
200879,2017-09-07 08:30:00+00:00,0.0,7,111.43,21.77,27.83,3.58,16.54


--> No more outliers in Yt, Rt and Ba

Now we can split our data into test and training data for our Support Vector Regression Model

In [44]:
# splitting the data
df = df.sort_values(by="Date_time")

x = df[["Ba_avg","day","Rt_avg","Yt_avg","Ws_avg","Ot_avg"]]
y = df['P_avg']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42) #80-20 split


In [45]:
from sklearn.svm import LinearSVR
svr = LinearSVR(random_state = 25)

In [ ]:
from sklearn.svm import LinearSVR
from sklearn import svm
svr = svm.LinearSVR(random_state=42, tol=1e-5, verbose=1, max_iter=10000)
svr = svr.fit(x_train, y_train)

[LibLinear]

In [ ]:
SVR_predict = svr.predict(x_test.values)

In [ ]:
twentyPecentcolumns = data.iloc[int(-43128):]

In [ ]:
plt.plot(x_test.values, y_test.values, color = 'blue', alpha=0.1)
plt.plot(x_test.values, SVR_predict, color = 'red', alpha=0.1)
plt.title('comparison of real data (blue) and predicted data (red)')
plt.show()

In [ ]:
plt.rc('font', size=12)
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(twentyPecentcolumns["Date_time"], y_test.values, color='tab:orange', label='Windpower real')
ax.plot(twentyPecentcolumns["Date_time"], SVR_predict, color='tab:blue', label='Windpower predicted')
ax.set_xlabel('Time')
ax.set_ylabel('Windpower')
ax.set_title('')
ax.grid(True)
ax.legend(loc='upper left');

In [ ]:
print('Coefficients:', svr.coef_)
y_pred = abs(SVR_predict) 
print("MAE: {}".format(np.abs(y_test-SVR_predict).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-SVR_predict)**2).mean())))
from sklearn.metrics import r2_score
r2 = r2_score(y_test.values, SVR_predict)
print("r2: {}".format(r2))

In [ ]:
# compute the SHAP values for the linear model
explainer = shap.LinearExplainer(svr, x_test)
shap_values = explainer.shap_values(x_test)
shap.summary_plot(shap_values, x, plot_type="bar")

smaller training data for a quicker training:

In [37]:
x_trains, x_tests, y_trains, y_tests = train_test_split(x, y, test_size = 0.8, random_state = 42)


In [38]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(x_trains.values)
yres = y_trains.values.reshape(-1,1)
y = sc_y.fit_transform(yres)
y = y.flatten()

In [39]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel="rbf")
svr_lin = SVR(kernel="linear")
svr_poly = SVR(kernel="poly")

In [40]:
rbf_model = svr_rbf.fit(x_train,y_train)

In [41]:
lin_model = svr_lin.fit(x_train,y_train)

In [42]:
poly_model = svr_poly.fit(x_train,y_train)

In [43]:
rbf_predict = rbf_model.predict(x_test.values)


X does not have valid feature names, but SVR was fitted with feature names


In [44]:
lin_predict = lin_model.predict(x_test.values)

X does not have valid feature names, but SVR was fitted with feature names


In [45]:
poly_predict = poly_model.predict(x_test.values)

X does not have valid feature names, but SVR was fitted with feature names


RBF model:

In [46]:
#print('Coefficients:', rbf_model.coef_)
y_pred = abs(rbf_predict) 
print("MAE: {}".format(np.abs(y_test-rbf_predict).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-rbf_predict)**2).mean())))
from sklearn.metrics import r2_score
r2 = r2_score(y_test.values, rbf_predict)
print("r2: {}".format(r2))

MAE: 173.8468019747961
RMSE: 281.8524601967515
r2: 0.6207875945091801


linear model:

In [47]:
print('Coefficients:', lin_model.coef_)
y_pred = abs(lin_predict) 
print("MAE: {}".format(np.abs(y_test-lin_predict).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-lin_predict)**2).mean())))
from sklearn.metrics import r2_score
r2 = r2_score(y_test.values, lin_predict)
print("r2: {}".format(r2))

Coefficients: [[  6.15364616  -0.19465732   2.91381276  -4.93024619 193.0716583
   -2.4919265 ]]
MAE: 116.02663678530611
RMSE: 209.89559068875604
r2: 0.7896969810513124


poly model:

In [48]:
#print('Coefficients:', poly_model.coef_)
y_pred = abs(poly_predict) 
print("MAE: {}".format(np.abs(y_test-poly_predict).mean()))
print("RMSE: {}".format(np.sqrt(((y_test-poly_predict)**2).mean())))
from sklearn.metrics import r2_score
r2 = r2_score(y_test.values, poly_predict)
print("r2: {}".format(r2))

MAE: 174.30119867346536
RMSE: 316.77094548766763
r2: 0.5210065682974265
